In [ ]:
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

### 15.1.1 Generate 100 random samples

In [ ]:
def gen_random(start=None, n=100):
    means = [20, 5]
    covs = [[2, 0.8], [0.8, 0.5]]
    return pd.DataFrame(stats.multivariate_normal.rvs(means, covs, size=n), columns=['ut', 'uc']), {}

In [ ]:
rand_data, rand_stats = gen_random()

In [ ]:
g = sns.jointplot(data=rand_data, x='ut', y='uc') 
g.ax_marg_x.axvline(rand_data.ut.mean(), color='r', linestyle='--')
g.ax_marg_y.axhline(rand_data.uc.mean(), color='r', linestyle='--')

In [ ]:
plt.plot(rand_data.ut, rand_data.uc, '.-')

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 6))
sns.histplot(rand_data, bins=50, stat='count')
ax.axvline(rand_data.ut.mean(), color='r', linestyle='--')
ax.axvline(rand_data.uc.mean(), color='r', linestyle='--')

### 15.1.2 Code up a Random Walk Metropolis sampler

In [ ]:
def propose_step(current):
    return stats.multivariate_normal.rvs(current)

In [ ]:
def calc_ratio(current, proposed):
    p_current = stats.multivariate_normal.pdf(current, mean=[20, 5], cov=[[2, 0.8], [0.8, 0.5]])
    p_proposed = stats.multivariate_normal.pdf(proposed, mean=[20, 5], cov=[[2, 0.8], [0.8, 0.5]])
    return p_proposed/p_current

In [ ]:
def get_next_value(current):
    proposed = propose_step(current)
    ratio = calc_ratio(current, proposed)
    u = stats.uniform.rvs()
    if ratio > u:
        return proposed, 1
    return current, 0

In [ ]:
def RWMetropolis(start, n):
    results = [start]
    num_accepted = 0
    for _ in range(n):
        next_step, accepted = get_next_value(results[-1])
        results.append(next_step)
        num_accepted += accepted
    stats = {'steps': n, 'acc_steps': num_accepted, 'perc_accepted': num_accepted/n}
    return pd.DataFrame(results, columns=['ut', 'uc']), stats

In [ ]:
rmw_data, rmw_stats = RWMetropolis((10, 5), 100)

In [ ]:
g = sns.jointplot(data=rmw_data, x='ut', y='uc') 
g.ax_marg_x.axvline(rmw_data.ut.mean(), color='r', linestyle='--')
g.ax_marg_y.axhline(rmw_data.uc.mean(), color='r', linestyle='--')

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 6))
sns.histplot(rmw_data, bins=50, stat='count')
ax.axvline(rmw_data.ut.mean(), color='r', linestyle='--')
ax.axvline(rmw_data.uc.mean(), color='r', linestyle='--')

In [ ]:
plt.plot(rmw_data.ut, rmw_data.uc, '.-')

### 15.1.3 Code up a Random Walk Metropolis sampler

In [ ]:
rmw_stats

### 15.1.4 Calculate Gelman's R

In [ ]:
def calc_W(chains):
    '''Calculate within chain variance'''
    # chains is list of lists at the moment
    return np.mean([np.var(c, ddof=1) for c in chains])

In [ ]:
def calc_B(chains):
    '''Calculate between chain variance'''
    # chains is list of lists at the moment
    glob_mean = np.mean(chains)
    chain_means = [np.mean(c) for c in chains]
    n_chains = len(chains)
    t = len(chains[0])
    return np.sum((chain_means-glob_mean)**2)*t/(n_chains-1)

In [ ]:
def calc_R(chains):
    B = calc_B(chains)
    W = calc_W(chains)
    T = len(chains[0])
#     print(B, W, T)
    return np.sqrt((W + (B-W)/T)/W)

### 15.1.5 Start 8 chains at (20,5) with lengh of 5 and calc R

In [ ]:
def RWMetropolis_chains(n_chains, l_chain, start_init):
    chains = [RWMetropolis(start_init(), l_chain)[0] for _ in range(n_chains)]
    R_ut = calc_R([c.ut for c in chains])
    R_uc = calc_R([c.uc for c in chains])
    return R_ut, R_uc, chains

In [ ]:
def constant_initializer():
    return (20, 5)

In [ ]:
R_ut, R_uc, chains = RWMetropolis_chains(8, 5, constant_initializer)

In [ ]:
def chains_plot(chains):
    N = len(chains[0].ut)
    R_uts = [calc_R([c.ut[:i] for c in chains]) for i in range(N+1)]
    R_ucs = [calc_R([c.uc[:i] for c in chains]) for i in range(N+1)]
    
    f, ax = plt.subplots(2, 2, figsize=(12,10))
    ax[0][0].plot(list(zip(*[c.ut for c in chains])), '.-')
    ax[0][0].set_title(f'ut ($\hat{{R}}$={R_uts[-1]:.3})')
    ax[0][0].set_ylim((0, 35))
    ax[0][1].plot(list(zip(*[c.uc for c in chains])), '.-')
    ax[0][1].set_title(f'uc ($\hat{{R}}$={R_ucs[-1]:.3})')
    ax[0][1].set_ylim((0, 9))
    
    ax[1][0].plot(R_uts)
    ax[1][1].plot(R_ucs)
    for a in ax[1]:
        a.set_ylim([0, 4])
        a.axhline(1.1, color='r')
        
    uts_conv = np.argmax(np.array(R_uts)<1.1)
    ucs_conv = np.argmax(np.array(R_ucs)<1.1)
    
    ax[1][0].axvline(uts_conv, color='g')
    ax[1][1].axvline(ucs_conv, color='g')
    ax[1][0].set_title(f'converged at {uts_conv}')
    ax[1][1].set_title(f'converged at {ucs_conv}')

In [ ]:
a = chains_plot(chains)

### 15.1.6 Start 8 chains at random locations and length 100

In [ ]:
def random_initializer():
    means = [20, 5]
    covs = np.identity(2)*40
    return stats.multivariate_normal.rvs(means, covs, size=1)

In [ ]:
R_ut, R_uc, chains = RWMetropolis_chains(8, 300, random_initializer)

In [ ]:
chains_plot(chains)

### 15.1.7 After how many iterations converged?

Convergence marked in green in above plots; it tooks about 250 iterations for ut and 150 for uc

### 15.1.8 Create Gibbs sampler

In [ ]:
def get_next_gibbs(ut, uc):
    new_ut = stats.norm.rvs(20 + 1.6*(uc-5), (1-0.8)**2*2)
    new_uc = stats.norm.rvs(5 + 0.4*(new_ut-20), (1-0.8)**2*0.5)
    return new_ut, new_uc

In [ ]:
def gibbs_sampler(start, l_chain):
    results = [start]
    for i in range(l_chain):
        new_sample = get_next_gibbs(*results[-1])
        results.append(new_sample) 
    stats = {'steps': l_chain}
    return pd.DataFrame(results, columns=['ut', 'uc']), stats

### 15.1.9 Draw 100 samples using Gibbs sampler

In [ ]:
gibbs_data, gibbs_stats = gibbs_sampler((10, 5), 100)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 6))
sns.histplot(rmw_data, bins=50, stat='count')
ax.axvline(gibbs_data.ut.mean(), color='r', linestyle='--')
ax.axvline(gibbs_data.uc.mean(), color='r', linestyle='--')

In [ ]:
plt.plot(gibbs_data.ut[50:], gibbs_data.uc[50:], '.-')

### 15.1.10 Draw 200 samples using different methods and compare error

In [ ]:
def get_error_estimates(algo, l_chain, warmup, n):
    data = [algo(random_initializer(), l_chain)[0][warmup:].ut for _ in range(n)]
    return [d.mean() - 20 for d in data]
        

In [ ]:
gibbs_errors = get_error_estimates(gibbs_sampler, 200, 100, 40)
rwm_errors = get_error_estimates(RWMetropolis, 200, 100, 40)
ind_errors = get_error_estimates(gen_random, 200, 100, 40)

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot([gibbs_errors, rwm_errors, ind_errors], bins=50)
plt.legend(['gibbs', 'RWM', 'indepedent'])
# sns.histplot(rwm_errors)

### 15.1.11 Average error vs sample size

In [ ]:
ns = list(range(5, 200, 5))
gibbs_error_avg = [np.mean(np.abs(get_error_estimates(gibbs_sampler, n, 0, 50))) for n in ns]
rwm_error_avg = [np.mean(np.abs(get_error_estimates(RWMetropolis, n, 0, 50))) for n in ns]
ind_error_avg = [np.mean(np.abs(get_error_estimates(gen_random, n, 0, 50))) for n in ns]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(ns, gibbs_error_avg, label='gibbs')
plt.plot(ns, rwm_error_avg, label='RWM')
plt.plot(ns, ind_error_avg, label='independent')
plt.legend()

Why Gibbs is better then independent if using 50% warmup?

### 15.1.12 Effective sample size for n = 150

In [ ]:
ind_150 = ns.index(150)
gibbs_150 = gibbs_error_avg[ind_150]
ns[np.argmax(ind_error_avg < gibbs_150)]

In [ ]:
rwm_150 = rwm_error_avg[ind_150]
ns[np.argmax(ind_error_avg < rwm_150)]

In [ ]:
rwm_error_avg[ind_150]